In [68]:
import pandas as pd

def read_DataFrame_from_file(filename: str, numberOfRows: int = None):
    return pd.read_excel(filename, nrows = numberOfRows, keep_default_na=False)

In [88]:
import re

DATA_INPUT_FILENAME = 'training_data.xlsx'
NUMBER_OF_PARSABLE_RECORDS = 999

def preprocess_data(data: pd.DataFrame):
    # data['person_address'] = data.apply(lambda row: re.sub(r'([^\s])(,)([^\s])', r'\1, \3', row['person_address']), axis=1)
    for col in data.columns:
        data[col] = data.apply(lambda row: re.sub(r'([^\s])(,)([^\s])', r'\1, \3', str(row[col])), axis=1)

# raw_data: pd.DataFrame = read_DataFrame_from_file(DATA_INPUT_FILENAME, NUMBER_OF_PARSABLE_RECORDS)
raw_data: pd.DataFrame = read_DataFrame_from_file('label.xlsx', NUMBER_OF_PARSABLE_RECORDS)

preprocess_data(raw_data)

In [100]:
token_types: set = {'co', 'building', 'street', 'nr', 'area', 'postal', 'city', 'region', 'country'}


def get_entity_list_0(entry: dict, address: str):
    entities: list = []
    present_tokens = filter(lambda item: item[0] in token_types and item[1] and str(item[1]).strip(), entry.items())

    for item in present_tokens:
        token_value = str(item[1]).strip()
        match = re.search(re.escape(token_value), address)
        if match:
            span = match.span()
            entities.append((span[0], span[1], item[0]))
        else:
            # Try and resolve multiple tokens separated by ';'
            split_items = map(lambda token: token.strip(), token_value.split(';'))
            for token in split_items:
                split_match = re.search(re.escape(token), address)
                if split_match:
                    span = split_match.span()
                    entities.append((span[0], span[1], item[0]))
                else:
                    print('WARNING: could not find token "{}" in address "{}"'.format(token, address))
    
    return entities

def get_entity_list(entry: dict, address: str):
    # print(entry)
    entities: list = []
    present_tokens = filter(lambda item: item[0] in token_types and item[1] and str(item[1]).strip(), entry.items())
    test = address
    for item in present_tokens:
        token_value = str(item[1]).strip()
        match = re.search(re.escape(token_value), test)
        if match:
            span = match.span()
            fromEntry = address[span[0]:span[1]]
            times = address.count(fromEntry)
            if times > 1:
                tkn = address.split()
                # print('tkn: ', type(tkn))
                # print(fromEntry, ' - count times: ', times, ' item:', item, ' add: ', address)

                size = -1
                for i in tkn:
                    if str(i).endswith(str(item[1])) and str(i).startswith(str(item[1])):
                        firstIdx = size + 1
                        lastIdx = firstIdx + len(str(i)) - 1
                        # print('item: ', item[1], ' first: ', firstIdx, ' last: ', lastIdx)
                        entities.append((firstIdx, lastIdx, item[0]))

                    string = str(i)
                    length = len(string) - 1
                    size = size + length + 2
                    # print(address, 'size: ', size, ' len: ', len(string))


            else:
                entities.append((span[0], span[1], item[0]))
        else:
            # Try and resolve multiple tokens separated by ';'
            split_items = map(lambda token: token.strip(), token_value.split(';'))
            for token in split_items:
                split_match = re.search(re.escape(token), test)
                if split_match:
                    span = split_match.span()
                    fromEntry = address[span[0]:span[1]]
                    times = address.count(fromEntry)
                    if times > 1:
                        tkn = address.split()
                        # print('tkn: ', type(tkn))
                        # print(fromEntry, ' - count times: ', times, ' item:', item, ' add: ', address, ' alone:')

                        size = -1
                        for i in tkn:
                            if str(i).endswith(str(item[1])) and str(i).startswith(str(item[1])):
                                firstIdx = size + 1
                                lastIdx = firstIdx + len(str(i)) - 1
                                # print('item: ', item[1], ' first: ', firstIdx, ' last: ', lastIdx)
                                entities.append((firstIdx, lastIdx, item[0]))

                            string = str(i)
                            length = len(string) - 1
                            size = size + length + 2
                            # print(address, 'size: ', size, ' len: ', len(string))

                    else:
                        entities.append((span[0], span[1], item[0]))
                    # entities.append((span[0], span[1], item[0]))
                else:
                    # address.replace(token,'',1)
                    print('WARNING: could not find token "{}" in address "{}"'.format(token, address))

    return entities


def get_entity_list_2(entry: dict, adr: str):
    address = str(adr)
    entities: list = []
    present_tokens = filter(lambda item: item[0] in token_types and item[1] and str(item[1]).strip(), entry.items())

    ## tokens to retry matching
    retry_tokens: set = set()

    for item in present_tokens:
        token_value = str(item[1]).strip()
        match = re.search(re.escape(token_value), address)
        if match:
            if (len(re.findall(re.escape(token_value), address)) > 1):
                # print('DEBUG: multiple occurences of {} in {}. Putting in retry queue'.format(token_value, address))
                retry_tokens.add((token_value, item[0]))
                continue
            span = match.span()
            entities.append((span[0], span[1], item[0]))
            address = address[:span[0]] + '$' * (span[1] - span[0]) + address[span[1]:]
        else:
            # Try and resolve multiple tokens separated by ';'
            split_items = map(lambda token: token.strip(), token_value.split(';'))
            for token in split_items:
                split_match = re.search(re.escape(token), address)
                if split_match:
                    if (len(re.findall(re.escape(token), address)) > 1):
                        # print('DEBUG: multiple occurences of {} in {}. Putting in retry queue'.format(token, address))
                        retry_tokens.add((token, item[0]))
                        continue
                    span = split_match.span()
                    entities.append((span[0], span[1], item[0]))
                    address = address[:span[0]] + '$' * (span[1] - span[0]) + address[span[1]:]
                else:
                    print('WARNING: could not find token "{}" in address "{}" | original {}'.format(token, address, adr))
    
    for token, tkn_type in retry_tokens:
        token_value = str(token).strip()
        match = re.search(re.escape(token_value), address)
        if match:
            # print('RESOLVED {} {} in {} AFTER RETRY | original {}'.format(tkn_type, token, address, adr))
            span = match.span()
            entities.append((span[0], span[1], tkn_type))
            address = address[:span[0]] + '$' * (span[1] - span[0]) + address[span[1]:]
        else:
            print('WARNING: could not find token "{}" in address "{}" | original {}'.format(token, address, adr))

    # if (len(retry_tokens)):
    #     print('TOKENS TO RETRY: {} | {}'.format(retry_tokens, address))

    return entities


def map_to_training_entry(entry: dict):
    address = entry['person_address']
    # print(address, {
    #     'entities': get_entity_list(entry, address)
    # })
    return (address, {
        'entities': get_entity_list(entry, address)
    })

def map_to_training_entry_2(entry: dict):
    address = entry['person_address']
    # print(address, {
    #     'entities': get_entity_list(entry, address)
    # })
    return (address, {
        'entities': get_entity_list_2(entry, address)
    })

def map_to_training_entry_0(entry: dict):
    address = entry['person_address']
    # print(address, {
    #     'entities': get_entity_list(entry, address)
    # })
    return (address, {
        'entities': get_entity_list_0(entry, address)
    })

train_data = list(
    map(map_to_training_entry, raw_data.to_dict('records'))
)


In [101]:
def entities_overlap(entry):
    entities = entry[1]['entities']
    for first in entities:
        for second in entities:
            if (first == second): continue
            if (first[0] < second[0] and first[1] > second[0]) or (first[0] > second[0] and first[1] < second[0]) or (first[0]==second[0] or first[1]==second[1]):
                print('Entities {} and {} overlap in "{}"'.format(first, second, entry[0]))
                return True
    return False

train_data = list(filter(lambda entry: not entities_overlap(entry), train_data))

Entities (25, 33, 'area') and (25, 30, 'region') overlap in "2-15, Meiwadori 3-chome, Hyogo-ku, Kobe-shi, Hyogo 652-0882"
Entities (35, 44, 'area') and (35, 40, 'city') overlap in "1-201, Fukuzaki 3-chome, Minato-ku Osaka-shi, Osaka 552-0013"
Entities (14, 34, 'building') and (18, 28, 'area') overlap in "Hitachi Ltd., New Marunouchi Bldg., 5-1, Marunouchi, 1-chome, Chiyoda-ku, Tokyo 100-8220"
Entities (36, 45, 'city') and (36, 41, 'region') overlap in "12-4, Sagisu 5-chome, Fukushima-ku, Osaka-shi, Osaka; 5530 002"
Entities (50, 60, 'city') and (50, 56, 'region') overlap in "c/o Shinko El. Ind. Co., Ltd. 80, Oshimada-machi, Nagano-shi Nagano 381-2287"
Entities (44, 57, 'city') and (44, 53, 'city') overlap in "c/o JMS CO. LTD., 12-17 Kakomachi, Naka-ku, Hiroshima-shi, Hiroshima 730-8652"
Entities (14, 34, 'area') and (14, 18, 'area') overlap in "4 NORMAN WAY, OVER INDUSTRIAL PARK, OVER, CAMBRIDGESHIRE"
Entities (34, 43, 'area') and (34, 39, 'city') overlap in "5-1-4-611, Shiginonishi, J

In [102]:
import spacy
import random

nlp_3 = spacy.blank('en')
ner_3 = nlp_3.create_pipe('ner')
nlp_3.add_pipe(ner_3)

for token in token_types:
    ner_3.add_label(token)

optimizer = nlp_3.begin_training()
for itn in range(1):
    random.shuffle(train_data)
    losses = {}

    batches = spacy.util.minibatch(train_data, size=spacy.util.compounding(4, 32, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp_3.update(
            texts,  
            annotations,  
            drop=0.5,  
            sgd=optimizer,
            losses=losses)

C:\Users\Home\anaconda3\envs\nlp-deps\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "28 Fayette Street, G-Unit, Cambrige MA 02139" with entities "[(3, 17, 'street'), (0, 2, 'nr'), (19, 25, 'area')...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\Home\anaconda3\envs\nlp-deps\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "1621 Carlton Drive, Parsippany, NJ 07054" with entities "[(5, 18, 'street'), (0, 4, 'nr'), (36, 40, 'postal...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\Home\anaconda3\envs\nlp-deps\lib\site-packages\spacy\language.py:

In [172]:
import spacy
import random

nlp = spacy.blank('en')
ner = nlp.create_pipe('ner')
nlp.add_pipe(ner)

for token in token_types:
    ner.add_label(token)

optimizer = nlp.begin_training()
for itn in range(20):
    random.shuffle(train_data)
    losses = {}

    batches = spacy.util.minibatch(train_data, size=spacy.util.compounding(4, 32, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(
            texts,  
            annotations,  
            drop=0.5,  
            sgd=optimizer,
            losses=losses)

C:\Users\Home\anaconda3\envs\nlp-deps\lib\site-packages\spacy\language.py:635: UserWarning: [W033] Training a new parser or NER using a model with an empty lexeme normalization table. This may degrade the performance to some degree. If this is intentional or this language doesn't have a normalization table, please ignore this warning.
  proc.begin_training(
C:\Users\Home\anaconda3\envs\nlp-deps\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "No. 66, Lane 642, Tung Wan Shou Rd. Gueishan Hsian..." with entities "[(18, 34, 'building'), (7, 16, 'street'), (0, 6, '...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\Home\anaconda3\envs\nlp-deps\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "MEDIZINISCHE UNIVER

In [173]:
nlp.to_disk('trained_model_0')
nlp2 = spacy.load('./trained_model_0')

for text, _ in train_data:
    doc = nlp2(text)
    print('{} Entities'.format(text), [(ent.text, ent.label_) for ent in doc.ents])

[('ner', <spacy.pipeline.pipes.EntityRecognizer object at 0x0000014F4F46C9A0>)]
[('ner', <spacy.pipeline.pipes.EntityRecognizer object at 0x0000014F58651AC0>)]
HSINCHU CITY Entities [('HSINCHU CITY', 'city')]
c/o Asahi Glass Company, Limited, 5-1, Marunouchi 1-chome, Chiyoda-ku, Tokyo 100-8405 Entities [('c/o Asahi Glass Company', 'co'), ('Limited', 'city'), ('5-1', 'street'), ('Marunouchi 1-chome', 'street'), ('Chiyoda-ku', 'area'), ('Tokyo', 'city'), ('100-8405', 'postal')]
920 EAST CREEK DRIVE, DRIPPING SPRINS, TEXAS 78620, E.U.A. US Entities [('920', 'nr'), ('EAST CREEK DRIVE', 'street'), ('DRIPPING SPRINS', 'city'), ('TEXAS', 'region'), ('78620', 'postal')]
Standartizatsii i Sertifikatsii Veterinarnôh Preparatov, Zvenigorodskoje u. 5 , Moskva, RUSSIAN FEDERATION Entities [('Standartizatsii i Sertifikatsii Veterinarnôh Preparatov', 'co'), ('Zvenigorodskoje u.', 'street'), ('5', 'nr'), ('Moskva', 'city'), ('RUSSIAN FEDERATION', 'region')]
2-2, MINAMIDAMACHI-2-CHOME, TOYAMA-SHI Entit

In [ ]:
from spacy.gold import GoldParse
from spacy.scorer import Scorer

def map_to_evaluation_model(entry: tuple):
    return (entry[0], entry[1]['entities'])


def evaluate(ner_model, examples):
    scorer = Scorer()
    for input_, annot in examples:
        doc_gold_text = ner_model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot)
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
    return scorer.scores

results = evaluate(nlp, map(map_to_evaluation_model, train_data))
print(results)

ModuleNotFoundError: No module named 'spacy.gold'